In [1]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
import re
import nltk
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string

data = pd.read_csv("twitter.csv")
data.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [3]:
data["labels"] = data["class"].map({0: "Hate Speech", 1: "Offensive Language", 2: "No Hate and Offensive"})
data.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet,labels
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,No Hate and Offensive
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,Offensive Language
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,Offensive Language
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,Offensive Language
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,Offensive Language


In [4]:
data = data[["tweet", "labels"]]
data.head()

,tweet,labels
0,!!! RT @mayasolovely: As a woman you shouldn't...,No Hate and Offensive
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,Offensive Language
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,Offensive Language
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,Offensive Language
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,Offensive Language


In [5]:
nltk.download('stopwords')
stopword=set(stopwords.words('english'))

def clean(text):
    text = str(text).lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>+', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text
data["tweet"] = data["tweet"].apply(clean)
data.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,tweet,labels
0,rt mayasolov woman shouldnt complain clean ho...,No Hate and Offensive
1,rt boy dat coldtyga dwn bad cuffin dat hoe ...,Offensive Language
2,rt urkindofbrand dawg rt ever fuck bitch sta...,Offensive Language
3,rt cganderson vivabas look like tranni,Offensive Language
4,rt shenikarobert shit hear might true might f...,Offensive Language


In [6]:


x = np.array(data["tweet"])
y = np.array(data["labels"])

cv = TfidfVectorizer(
    ngram_range=(1, 2),   # unigrams + bigrams
    max_df=0.9,          # ignore very common words
    min_df=3             # ignore extremely rare words
)

X = cv.fit_transform(x)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
)

In [7]:
clf = LogisticRegression(max_iter=1000, class_weight="balanced")
clf.fit(X_train,y_train)
clf.score(X_test,y_test)
user = input("Enter a Text: ")
data = cv.transform([user]).toarray()
output = clf.predict(data)
print(output)

Enter a Text:  what's up my man


['Offensive Language']


In [8]:
print("Train Score:", clf.score(X_train, y_train))
print("Test Score:", clf.score(X_test, y_test))


Train Score: 0.9152613827993255
Test Score: 0.862697151240983


In [9]:


y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


                       precision    recall  f1-score   support

          Hate Speech       0.34      0.57      0.43       465
No Hate and Offensive       0.75      0.93      0.83      1379
   Offensive Language       0.97      0.87      0.92      6335

             accuracy                           0.86      8179
            macro avg       0.69      0.79      0.72      8179
         weighted avg       0.89      0.86      0.87      8179



In [13]:
svc = LinearSVC(class_weight="balanced")
svc.fit(X_train, y_train)

print("LinearSVC Train Score:", svc.score(X_train, y_train))
print("LinearSVC Test Score:", svc.score(X_test, y_test))

y_pred_svc = svc.predict(X_test)
print(classification_report(y_test, y_pred_svc))

LinearSVC Train Score: 0.9821729703685859
LinearSVC Test Score: 0.8829930309328768
                       precision    recall  f1-score   support

          Hate Speech       0.39      0.34      0.36       465
No Hate and Offensive       0.79      0.87      0.83      1379
   Offensive Language       0.94      0.93      0.93      6335

             accuracy                           0.88      8179
            macro avg       0.71      0.71      0.71      8179
         weighted avg       0.88      0.88      0.88      8179

